# Sample self Attention implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(1337)
B,T,C=3,8,32 ## Batch time channels/embed_dim
x=torch.randn(B,T,C)
head_size=16 ## Head size is a hyperparameter
## Information at time T is combined to produce key and query vector
key=nn.Linear(C,head_size,bias=False) ## B,T,head_size
query=nn.Linear(C,head_size,bias=False)##B,T,head_size
value=nn.Linear(C,head_size,bias=False)
k=key(x)
q=query(x)
v=value(x)
wei=q@k.transpose(-1,-2) ## k made head_size*T before broadcast ## wei is B,T,T
tril=torch.tril(torch.ones(T,T))## Lower triangular matrix of all ones
## The masking is not present in decoder blocks but no masking is needed in 
wei=wei.masked_fill(tril==0,float("-inf"))/(head_size**0.5)# Scaled self attention
wei=F.softmax(wei,dim=-1)
out=wei@v ## B*T*C